In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Directories of dataset
train_dir = '../data/food11/train'
test_dir = '../data/food11/test'

# Define ImageDataGenerator for preprocessing and augmentation
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Load train and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 9900 images belonging to 11 classes.
Found 1100 images belonging to 11 classes.


In [9]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load MobileNet without top layers
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(11, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [10]:
# Train the model on the train dataset
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=test_generator,
    validation_steps=test_generator.samples // 32,
    epochs=30
)

# Save the trained model
model.save('mobilenet_food11.h5')

Epoch 1/30
309/309 [==============================] - 110s 353ms/step - loss: 1.1226 - accuracy: 0.6472 - val_loss: 0.7268 - val_accuracy: 0.7721
Epoch 2/30
309/309 [==============================] - 113s 366ms/step - loss: 0.7354 - accuracy: 0.7652 - val_loss: 0.6117 - val_accuracy: 0.7950
Epoch 3/30
309/309 [==============================] - 114s 369ms/step - loss: 0.6454 - accuracy: 0.7931 - val_loss: 0.5644 - val_accuracy: 0.8199
Epoch 4/30
309/309 [==============================] - 239s 773ms/step - loss: 0.6055 - accuracy: 0.8058 - val_loss: 0.5459 - val_accuracy: 0.8290
Epoch 5/30
309/309 [==============================] - 204s 658ms/step - loss: 0.5670 - accuracy: 0.8160 - val_loss: 0.5719 - val_accuracy: 0.8143
Epoch 6/30
309/309 [==============================] - 110s 356ms/step - loss: 0.5426 - accuracy: 0.8230 - val_loss: 0.5297 - val_accuracy: 0.8336
Epoch 7/30
309/309 [==============================] - 114s 369ms/step - loss: 0.5156 - accuracy: 0.8327 - val_loss: 0.5018 -

In [11]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // 32)
print(f'Test Accuracy: {test_acc}, Test Loss: {test_loss}')

34/34 [==============================] - 8s 241ms/step - loss: 0.5129 - accuracy: 0.8428
Test Accuracy: 0.8428308963775635, Test Loss: 0.5128651261329651
